## Environment Setting

In [ ]:
# install dependencies: 
!pip install pyyaml==3.12 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

1.8.1+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# After running this module, it is essential to running all modules again to successfully install the detectron libraries
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
torch.__version__

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
     |████████████████████████████████| 5.7MB 687kB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 645kB 15.2MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=824716456c7b0fb7d44867bd22a2c2cf2b36702465015c28ca3c51fdacba2f0d
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
Successfully built fvcore
  Found existing installation: PyYAML 3.12
    Uninstalling PyYAML-3.12:
      Successfully uninstalled PyYAML-3.12


'1.8.1+cu101'

In [ ]:
# Clone Detectron and PubLayNet tools from created Github
! git clone https://github.com/Dean930610/Detectron_Publayout.git

Cloning into 'Detectron_Publayout'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1493 (delta 13), reused 0 (delta 0), pack-reused 1462
Receiving objects: 100% (1493/1493), 34.80 MiB | 24.82 MiB/s, done.
Resolving deltas: 100% (870/870), done.


## Acquire the pre-trained model by PubLayNet

In [ ]:
# Code to download Pub
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Mask-RCNN model
id = '1aKWqLAIMRDEq5lERVY_s_uVw5W1984z1'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('final_model.pkl')  

## Essential Liraries installing

In [ ]:
import sys
import requests
import tarfile
import json
import numpy as np
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
%matplotlib inline
!nvidia-smi
import tensorflow as tf
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

Fri Apr  9 23:50:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

'/device:GPU:0'

In [ ]:
# Detectron Installing
! Clone Detectron properties
! git clone https://github.com/Dean930610/Detectron_Publayout.git
! pip install -r /content/Detectron_Publayout/requirements.txt
! cd /content/Detectron_Publayout && make

/bin/bash: Clone: command not found
fatal: destination path 'Detectron_Publayout' already exists and is not an empty directory.
Processing /root/.cache/pip/wheels/03/05/65/bdc14f2c6e09e82ae3e0f13d021e1b6b2481437ea2f207df3f/PyYAML-3.12-cp37-cp37m-linux_x86_64.whl
ERROR: omegaconf 2.0.6 has requirement PyYAML>=5.1.*, but you'll have pyyaml 3.12 which is incompatible.
ERROR: fvcore 0.1.3.post20210317 has requirement pyyaml>=5.1, but you'll have pyyaml 3.12 which is incompatible.
  Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1


python setup.py develop --user
Compiling detectron/utils/cython_bbox.pyx because it changed.
Compiling detectron/utils/cython_nms.pyx because it changed.
[1/2] Cythonizing detectron/utils/cython_bbox.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Detectron_Publayout/detectron/utils/cython_bbox.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2] Cythonizing detectron/utils/cython_nms.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Detectron_Publayout/detectron/utils/cython_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
creating Detectron.egg-info
writing Detectron.egg-info/PKG-INFO
writing dependency_links to Detectron.egg

# Generate COCO format Json (Actually PubLayNet format Json)

In [ ]:
import os
import json
import collections
import cv2 
 
coco = collections.OrderedDict()
coco['images'] = []
coco['type'] = 'instances'
coco['annotations'] = []
coco['categories'] = []

image_id = 420000 #id
category_item_id = 1
classname = [
         'text', 'title', 'list', 'table', 'figure']

In [ ]:
def GetFileFromThisRootDir(dir,ext = None):
  allfiles = []
  needExtFilter = (ext != None)
  for root,dirs,files in os.walk(dir):   
    for filespath in files:            #read file name 
      filepath = os.path.join(root, filespath)    #created abosolute path
      extension = os.path.splitext(filepath)[1][1:]  #os.path.splitext(path)  #分割路径，返回路径名和文件后缀 其中[1]为后缀.png，再取[1:]得到png
      if needExtFilter and extension in ext:
        allfiles.append(filepath)
      elif not needExtFilter:
        allfiles.append(filepath)
  return allfiles   #return the path


In [ ]:
def addCatItem(name):
    '''
    add json category part
    '''
    global category_item_id
    category_item = collections.OrderedDict()
    category_item['supercategory'] = 'none'
    category_item['id'] = category_item_id
    category_item['name'] = name
    coco['categories'].append(category_item)
    category_item_id += 1

In [ ]:
def addImgItem(img_name, size):
    '''
    add image part in to json
    '''
    global image_id
    if img_name is None:
        raise Exception('Could not find Picture file.')
    if size['width'] is None:
        raise Exception('Could not find width.')
    if size['height'] is None:
        raise Exception('Could not find height.')
    #image_item = dict()    #order collections.OrderedDict()
    image_item = collections.OrderedDict()
    image_item['file_name'] = img_name  
    image_item['width'] = size['width']   
    image_item['height'] = size['height']
    image_item['id'] = image_id
    coco['images'].append(image_item) 
    image_id = image_id+1


In [ ]:
def WriteCOCOFiles(pic_path):
    for idx,path in enumerate(pic_path):
        size = {}
        imgname = os.path.basename(path) #delete added name
        img = cv2.imread(path)
        size['height'] = img.shape[0] 
        size['width'] = img.shape[1]  
        addImgItem(imgname, size)
        print('add image with {} and {}'.format(imgname, size))
    for idx, obj in enumerate(classname):
        addCatItem(obj)    

In [ ]:

if __name__ == '__main__':
    pic_dir = '/content/publaynet/test'    
    json_file = '/content/test.json'  
    pic_path = GetFileFromThisRootDir(pic_dir,ext = None)  
    WriteCOCOFiles(pic_path)
json.dump(coco, open(json_file, 'w'))

# Generate Json File for Competition Submission


In [ ]:
!cp -ri /content/drive/MyDrive/tabel_extraction/* /content/Detectron_Publayout/detectron/datasets/data/test/

In [ ]:
!mv /content/test.json /content/Detectron_Publayout/detectron/datasets/data

In [ ]:
! cd /content/Detectron_Publayout;python /content/Detectron_Publayout/tools/test_net.py \
    --cfg /content/Detectron_Publayout/PubLayNet/PubLayNet-master/pre-trained-models/Mask-RCNN/e2e_mask_rcnn_X-101-64x4d-FPN_1x.yaml \
    TEST.WEIGHTS /content/final_model.pkl \

Found Detectron ops lib: /usr/local/lib/python3.7/dist-packages/torch/lib/libcaffe2_detectron_ops_gpu.so
[E init_intrinsics_check.cc:43] CPU feature avx is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:43] CPU feature avx2 is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:43] CPU feature fma is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
INFO test_net.py:  98: Called with args:
INFO test_net.py:  99: Namespace(cfg_file='/content/Detectron_Publayout/PubLayNet/PubLayNet-master/pre-trained-models/Mask-RCNN/e2e_mask_rcnn_X-101-64x4d-FPN_1x.yaml', multi_gpu_testing=False, opts=['TEST.WEIGHTS', '/content/final_model.pkl'], range=None, vis=False, wait=True)
INFO test_net.py: 105: Testing with config:
INF

# Results analysing

In [ ]:
import json
with open('/content/bbox_coco_2014_val_results.json') as f:
  results = json.load(f)


JSONDecodeError: ignored

In [ ]:
print(len(results))

41258


In [ ]:
print(results[24])

{'image_id': 0, 'category_id': 1, 'bbox': [236.99546813964844, 41.55669403076172, 154.5416717529297, 17.43746566772461], 'score': 0.07738644629716873}


In [ ]:
score = []
j = 0
for i in range(len(results)):
  score.append(results[i]['score'])
  if results[i]['score'] > 0.7:
    j += 1

In [ ]:
score = []
j = 0
for i in range(len(results)):
  score.append(results[i]['category_id'])
  if results[i]['category_id'] == 4 and results[i]['score'] > 0.7:
    j += 1

In [ ]:
print(j)

168
